In [1]:
import cobra
import pandas as pd
import numpy as np

In [2]:
new_model = cobra.io.read_sbml_model('Models/Recon2_Influenza_consistent.xml')

In [5]:
core_table=pd.read_csv('/csvs/Influenza_assembled.csv', sep=';')

In [6]:
print(len(core_table))
core_table.head()

8537


,Unnamed: 0,Mock_72h_A549,Mock_24h_A549,Mock_96h_A549,Mock_48h_A549,Mock_96h_Hep2,RSV_BA_48h_Hep2,RSV_BA_24h_Hep2,Mock_72h_Hep2,RSV_BA_24h_A549,...,RSV_18537_96h_Hep2,RSV_18537_24h_Hep2,RSV_ON1_24h_Hep2,RSV_ON1_96h_A549,RSV_ON1_48h_A549,RSV_ON1_72h_A549,RSV_ON1_96h_Hep2,RSV_ON1_24h_A549,RSV_ON1_48h_Hep2,RSV_ON1_72h_Hep2
0,13DAMPPOX,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
1,1a_24_25VITD2Hm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1a_24_25VITD3Hm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1a_25VITD2Hm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1a_25VITD3Hm,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
core_table = core_table.rename(columns={'Unnamed: 0': 'index'})
core_table.set_index('index', inplace=True)
core_table.head()

,Mock_72h_A549,Mock_24h_A549,Mock_96h_A549,Mock_48h_A549,Mock_96h_Hep2,RSV_BA_48h_Hep2,RSV_BA_24h_Hep2,Mock_72h_Hep2,RSV_BA_24h_A549,RSV_BA_48h_A549,...,RSV_18537_96h_Hep2,RSV_18537_24h_Hep2,RSV_ON1_24h_Hep2,RSV_ON1_96h_A549,RSV_ON1_48h_A549,RSV_ON1_72h_A549,RSV_ON1_96h_Hep2,RSV_ON1_24h_A549,RSV_ON1_48h_Hep2,RSV_ON1_72h_Hep2
index,,,,,,,,,,,,,,,,,,,,,
13DAMPPOX,1,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
1a_24_25VITD2Hm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1a_24_25VITD3Hm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1a_25VITD2Hm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1a_25VITD3Hm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
for column in core_table.columns:
    core_table.loc['biomass_reaction'][column] = 1

In [9]:
vbof_df = pd.DataFrame(data = [[1]* len(core_table.columns)],index = ['VBOF'],columns = core_table.columns)

In [10]:
core_table = core_table.append([vbof_df])
print(len(core_table))

8538


In [11]:
#Create a dictionary that directly retreives the reaction number based on the reaction-ID
useful_dict = {}
for i in range(len(new_model.reactions)):
    useful_dict[new_model.reactions[i].id] = i+1

In [12]:
#Counting single cells that don't have enough core reactions  
not_enough = []
solved = []

In [13]:
#Iterate over every reaction
for col in core_table.columns:
    if core_table[col].sum() > 30:
        column = col
        core_indices = []
        for index in core_table.index:
            try:
                if core_table.loc[index][column] == 1:
                    try:
                        core_indices.append(useful_dict[index])
                    except:
                        continue
            except:
                print(column, index)
        if len(core_indices) >= 30:
            with open('CoreReactsInfluenza/'+column+'.csv', 'w', newline='') as f:
                for item in core_indices:
                    f.write(str(item)+'\n')
        else:
            not_enough.append(column)
    else:
        not_enough.append(column)
    solved.append(column)